In [ ]:
#| label: daFig1jn

# Prepare Python environment

import scipy.io as sio
from pathlib import Path
import os
import sys
from contextlib import contextmanager
from repo2data.repo2data import Repo2Data

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    repo_path = Path(os.path.abspath('')).parents[2]
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / '..' / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
    if not dataset_path.exists():
        repo2data = Repo2Data(data_req_path)

data_dir = dataset_path / "04-B1-01-DoubleAngle"
data_file = "da_fig1.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

nom_first_ang = mat_contents["nom_first_ang"][0][0]
B1 = mat_contents["B1"][0]
TR_range = mat_contents["TR_range"][0]
B1_varTR = mat_contents["B1_varTR"]
linfit = mat_contents["linfit"]

## Plot

# Module imports

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

# PYTHON CODE

init_notebook_mode(connected=True)
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.

wm_data = [dict(
            visible = False,
            x = B1,
            y = B1_varTR[ii,:,0],
            name = "T1 = 900 ms (WM)",
            line=dict(color="blue", width=2),
            hoverinfo = "y") for ii in range(len(TR_range))]

wm_data[22]['visible'] = True


gm_data = [dict(
        visible = False,
        x = B1,
        y = B1_varTR[ii,:,1],
        name = "T1 = 1500 ms (GM)",
        line=dict(color="orange", width=2),
        hoverinfo = "y") for ii in range(len(TR_range))]

gm_data[22]['visible'] = True

csf_data = [dict(
        visible = False,
        x = B1,
        y = B1_varTR[ii,:,2],
        name = "T1 = 4000 ms (CSF)",
        line=dict(color="green", width=2),
        hoverinfo = "y") for ii in range(len(TR_range))]

csf_data[22]['visible'] = True

b1_data = [dict(
        visible = False,
        x = B1,
        y = B1,
        name = "B1 identity line",
        line=dict(color="black", width=1, dash = 'dash'),
        hoverinfo = "y") for ii in range(len(TR_range))]

b1_data[22]['visible'] = True

data = wm_data + gm_data + csf_data + b1_data

# Create and add slider
steps = []
      
for i in range(len(TR_range)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(b1_data)},],  # layout attribute
        label = str(TR_range[i])
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=22,
    currentvalue={"prefix": "TR (ms) = "},
    pad={"t": 50},
    steps=steps
)]

layout = go.Layout(
    width=600,
    height=600,
    margin=go.layout.Margin(
        l=100,
        r=80,
        b=100,
        t=130,
    ),
    annotations=[
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='Computed B1',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.50,
            y=-0.15,
            showarrow=False,
            text='Actual B1',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=0,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2,
        range=[0.7,1.3]
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='rgb(169,169,169)',
        linecolor='black',
        linewidth=2,
        range=[0.7,1.3]
    ),
    legend=dict(
        x=0.25,
        y=1.3,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'fig1.html', config = config)
